In [3]:
# Import all the necessary packages
import pandas as pd
import numpy as np

import statsmodels.api as sm
import scipy.stats as st

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns

In [4]:
# Plot style & size defaults (OPTIONAL)
rcParams['figure.figsize'] = (11.7,8.27)
rcParams['axes.labelsize'] = "x-large"

In [5]:
# Helper function for annotating the pair plots w/ corr coefficients.

def corr_annot(x, y, ax=None, **kws):
    """Annotate a plot w/ the correlation coefficient."""
    r, _ = st.pearsonr(x, y)
    ax = ax or plt.gca()
    fontsizes = {range(0,21):"medium", range(21,51):'large', range(51,81):'x-large', range(90,101):'xx-large'}

    if r > 0:
        font_size = [fontsize for bound,fontsize in fontsizes.items() if round(abs(r), 1)*100 in bound][0]
        ax.annotate(f'ρ = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes, fontsize = font_size)

### Read in the dataset and take a look at variable relations

In [ ]:
# Read in the dataset
df_squawk_model = pd.read_csv("../data/processed/squawk7700_model.csv")

In [ ]:
# Check the relationships among the explanatory variables

g = sns.pairplot(df_squawk_model.drop(columns = 'diversion'), 
                 kind = "reg",
                 corner = True,
                 plot_kws = {
                     'line_kws':{'color':'red'},
                     #'logistic':True
                     #'lowess':True
                     });

g.map_lower(corr_annot);

### Implement the logit-Binomial GLM model

In [ ]:
y = df_squawk_model['diversion']
X = df_squawk_model.drop(columns = 'diversion')

# with GLM (generalized linear model)
squawk_model_all = sm.GLM(
    y,
    sm.add_constant(X),
    family=sm.families.Binomial(
        link=sm.genmod.families.links.Logit()
        )
    ).fit()

print(squawk_model_all.summary())

# We can check the AIC to evaluate the model (lower is better)
print("AIC:", squawk_model_all.aic)

In [ ]:
with sns.plotting_context("notebook", font_scale=1.5):
    sns.regplot(data = df_squawk_model, x = 'engine', y= 'diversion', logistic=True, ci=95);

### Generalized Regression Model

In [6]:
from IPython.display import clear_output

def stepwise_selection(data,
                       pred_class,
                       initial_list=[], 
                       aic_step_min = 2, 
                       #pval_threshold = 0.05, 
                       verbose=True):
    """ Perform a forward feature selection 
    based on AIC values from statsmodels.api.GLM.
    """
    
    y = data[pred_class]
    X = data.drop(columns = pred_class)
    included = list(initial_list)
    
    # Forward step
    def forward_step(y, X, included):
      old_aic = 0
      while True:
        changed = False
        excluded = list(set(X.columns)-set(included))
        new_aic = pd.Series(index=excluded, dtype='float64')
        #print(new_aic)
        if old_aic==0:
            model_old = sm.GLM(y,
                            sm.add_constant(X[included]),
                                              family=sm.families.Binomial(
                                                  link=sm.genmod.families.links.Logit()
                                                  )
                                              ).fit(method='bfgs',maxiter=1000)
            old_aic = model_old.aic
        for new_column in excluded:

            model = sm.GLM(y,
                            sm.add_constant(X[included + [new_column]]),
                                              family=sm.families.Binomial(
                                                  link=sm.genmod.families.links.Logit()
                                                  )
                                              ).fit(method='bfgs',maxiter=1000)

            new_aic[new_column] = model.aic
        best_aic = new_aic.min()
        if old_aic-best_aic > aic_step_min and len(new_aic) > 1:
            clear_output(wait=True)
            best_feature = new_aic.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:20} Current  AIC: {:.6}'.format(best_feature, best_aic))
            old_aic = best_aic
        if not changed:
            break
      print("Best AIC: {:.6}".format(best_aic))
      return (best_aic, included, model)

    return forward_step(y, X, included)

In [ ]:
best_aic, features, model = stepwise_selection(df_squawk_model, pred_class = "diversion")
print(best_aic, features, model.params)

In [ ]:
print(f'Features used in best model (AIC: {best_aic}):')
print(features)
print(model.summary())
print(model.aic)